<a href="https://colab.research.google.com/github/sophieelin/RAG-for-Medical-Queries/blob/main/RAG_for_Medical_Queries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q langchain faiss-cpu huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 70.4 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login

# Log in to Hugging Face
login(token="redacted")

In [ ]:
#from transformers import AutoTokenizer, AutoModelForCausalLM
#import torch


#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
#model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B").to(device)



!pip install faiss-cpu
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pickle

embed_model = SentenceTransformer("all-MiniLM-L6-v2")

# Load
index = faiss.read_index("chunks.index")



# Test with a medical question
# query = "What are the symptoms of diabetes?"
# response = generate_medical_answer(query)
# print(response)



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 60.8 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Generate text for a medical query
def generate_medical_answer(query):
    inputs = tokenizer(query, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=512)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

In [ ]:
import json
with open('chunks.json', 'r') as json_file:
    texts = json.load(json_file)

with open('questions.json', 'r') as json_file:
    questions = json.load(json_file)

from openai import OpenAI
client = OpenAI(api_key="redacted")


# aug_query = "context: " + "Systematic testing and treatment for TB infection may be considered for prisoners, health workers, immigrants from countries with a high TB burden, homeless people and people who use drugs. (Conditional recommendation, low to very low certainty of the estimates of effect)" + " Question: " + query

# # response = generate_medical_answer(query)
# aug_response = generate_medical_answer(aug_query)
# # print(response)
# print(aug_response)

In [ ]:

for document in questions:
  for section in document:
    for question in document[section]:
      query = question['question'].split(':contentReference')[0]
      choices=question['choices']
      answer=question['answer']

      query_embedding = embed_model.encode([query])

      D, I = index.search(np.array(query_embedding), k=5)
      combined_texts = texts[I[0][0]]
      for i in range(1,len(I[0])):
        combined_texts += " " + texts[I[0][i]]

      aug_query = f"Given the following information to supplement your answer: {combined_texts} Answer the question: {query}"
      if section!='open_response_questions':
        query += f" Choices: {choices}. In your response, state the letter of the correct answer."
        aug_query += f" Choices: {choices} In your response, state the letter of the correct answer."

      response = client.responses.create(
          model="gpt-3.5-turbo",
          input=query
      )

      aug_response= client.responses.create(
          model="gpt-3.5-turbo",
          input=aug_query
      )

      with open(f"{section}.txt", "a") as f:
        f.write(f"Question: {query}\n\n")
        f.write(f"     {response.output_text}.\n\n\n")

      with open(f"{section}_rag.txt", "a") as f:
        f.write(f"Question: {query}\n\n")
        f.write(f"     {aug_response.output_text}.\n\n\n")